In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn google-generativeai groq python-dotenv tqdm -q

In [ ]:
# Suppress deprecation warnings
import warnings
import os

warnings.filterwarnings('ignore', category=FutureWarning, module='google.generativeai')

# Import libraries

# Fix SSL/TLS certificate verification for gRPC (required for Google Gemini API on macOS)
os.environ['GRPC_DEFAULT_SSL_ROOTS_FILE_PATH'] = ''
os.environ['GRPC_SSL_CIPHER_SUITES'] = 'HIGH'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
import ssl
from datetime import datetime
from tqdm import tqdm

# Fix SSL certificate verification issue on macOS
ssl._create_default_https_context = ssl._create_unverified_context

# API setup
import google.generativeai as genai
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure APIs
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
if GROQ_API_KEY:
    groq_client = Groq(api_key=GROQ_API_KEY)

# Set plot style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)

print("✓ Libraries imported successfully")
print("✓ SSL certificate verification disabled for macOS compatibility")
print(f"✓ Google API configured: {bool(GOOGLE_API_KEY)}")
print(f"✓ Groq API configured: {bool(GROQ_API_KEY)}")

## 1. Load Dataset

In [ ]:
# Load the 100% agreement dataset
data_path = "../../DatasetAnalysis_FinancialPhraseBank/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt"

sentences = []
sentiments = []

with open(data_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        line = line.strip()
        if "@" in line:
            parts = line.rsplit("@", 1)
            if len(parts) == 2:
                sentences.append(parts[0])
                sentiments.append(parts[1])

df = pd.DataFrame({"sentence": sentences, "sentiment": sentiments})

print(f"Dataset loaded: {len(df)} sentences")
print(f"\nSentiment distribution:")
print(df["sentiment"].value_counts())
display(df.sample(5, random_state=42))

## 2. Zero-Shot Insight Generation Prompt Design

**Prompt Strategy**: Generate comprehensive financial insights without examples.

**Insight Categories**:
1. **Financial Trend**: Direction and magnitude of change
2. **Business Impact**: Implications for operations and strategy
3. **Stakeholder Perspective**: Impact on investors, employees, customers
4. **Risk/Opportunity**: Potential concerns or advantages
5. **Recommended Actions**: Suggested next steps

In [ ]:
def create_zero_shot_insight_prompt(sentence):
    """
    Creates a zero-shot prompt for financial insight generation.
    No examples provided - model relies on pretrained knowledge.
    """
    prompt = f"""You are a financial analyst expert. Generate comprehensive, actionable insights from the following financial statement.

Financial Statement:
"{sentence}"

Provide detailed analysis covering:
1. **Key Trend**: What financial trend or change does this indicate?
2. **Business Impact**: What are the implications for the company's operations and strategy?
3. **Stakeholder Impact**: How does this affect investors, employees, and customers?
4. **Opportunities/Risks**: What opportunities or risks does this present?
5. **Recommended Actions**: What should management or investors consider?

Provide your response in the following JSON format:
{{
    "key_trend": "Concise description of the main trend",
    "business_impact": "Impact on company operations and strategy",
    "stakeholder_impact": "Effects on different stakeholder groups",
    "opportunities": "Potential opportunities identified",
    "risks": "Potential risks or concerns",
    "recommended_actions": "Suggested actions for management/investors",
    "insight_quality_score": 0.0-1.0
}}
"""
    return prompt


# Test prompt
test_sentence = "Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007."
print("=" * 80)
print("ZERO-SHOT INSIGHT GENERATION PROMPT EXAMPLE")
print("=" * 80)
print(create_zero_shot_insight_prompt(test_sentence))

## 3. Model Inference Functions

In [ ]:
def call_gemini(prompt, model_name="gemini-2.0-flash-exp", temperature=0.3):
    """Call Gemini API with retry logic - higher temperature for creativity"""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=temperature,
                    max_output_tokens=800,
                ),
            )
            return response.text
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2**attempt)
                continue
            return None
    return None


def call_llama(prompt, temperature=0.3):
    """Call Llama via Groq API"""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            chat_completion = groq_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama-3.3-70b-versatile",
                temperature=temperature,
                max_tokens=800,
            )
            return chat_completion.choices[0].message.content
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2**attempt)
                continue
            return None
    return None


def parse_insight_response(response_text):
    """Parse JSON response from model"""
    try:
        if "```json" in response_text:
            json_str = response_text.split("```json")[1].split("```")[0].strip()
        elif "```" in response_text:
            json_str = response_text.split("```")[1].strip()
        else:
            json_str = response_text.strip()

        result = json.loads(json_str)
        return result
    except:
        # If parsing fails, return None
        return None


print("✓ Model inference functions defined")

## 4. Run Experiments

### E23: Gemini 2.5 Pro (Zero-Shot Insights)

In [ ]:
# For testing, use a sample of the dataset
test_df = df.head(50).copy()  # Using 50 samples for insight generation

# E23: Gemini 2.5 Pro
print("Running E23: Gemini 2.5 Pro (Zero-Shot Insight Generation)...")
e23_results = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="E23 Progress"):
    prompt = create_zero_shot_insight_prompt(row["sentence"])
    response = call_gemini(prompt, model_name="gemini-2.0-flash-exp")

    if response:
        parsed = parse_insight_response(response)
        if parsed:
            e23_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["sentiment"],
                    "key_trend": parsed.get("key_trend", "N/A"),
                    "business_impact": parsed.get("business_impact", "N/A"),
                    "stakeholder_impact": parsed.get("stakeholder_impact", "N/A"),
                    "opportunities": parsed.get("opportunities", "N/A"),
                    "risks": parsed.get("risks", "N/A"),
                    "recommended_actions": parsed.get("recommended_actions", "N/A"),
                    "insight_quality_score": parsed.get("insight_quality_score", 0.5),
                }
            )
        else:
            e23_results.append(
                {
                    "sentence": row["sentence"],
                    "true_sentiment": row["sentiment"],
                    "key_trend": "Parse error",
                    "business_impact": "N/A",
                    "stakeholder_impact": "N/A",
                    "opportunities": "N/A",
                    "risks": "N/A",
                    "recommended_actions": "N/A",
                    "insight_quality_score": 0,
                }
            )

    time.sleep(1.0)  # Slower rate for longer responses

e23_df = pd.DataFrame(e23_results)
print(f"\n✓ E23 completed: {len(e23_df)} insights generated")
display(e23_df.head(3))

## 5. Analyze Insight Quality

In [ ]:
# Display detailed insights for review
def display_insight_details(df, num_samples=3):
    """Display formatted insights for qualitative review"""
    samples = df.sample(min(num_samples, len(df)), random_state=42)

    for idx, row in samples.iterrows():
        print("\n" + "=" * 100)
        print(f"STATEMENT: {row['sentence']}")
        print(f"TRUE SENTIMENT: {row['true_sentiment']}")
        print("=" * 100)
        print(f"\n📊 KEY TREND:\n{row['key_trend']}")
        print(f"\n💼 BUSINESS IMPACT:\n{row['business_impact']}")
        print(f"\n👥 STAKEHOLDER IMPACT:\n{row['stakeholder_impact']}")
        print(f"\n✅ OPPORTUNITIES:\n{row['opportunities']}")
        print(f"\n⚠️  RISKS:\n{row['risks']}")
        print(f"\n🎯 RECOMMENDED ACTIONS:\n{row['recommended_actions']}")
        print(f"\n⭐ QUALITY SCORE: {row['insight_quality_score']:.2f}")
        print("=" * 100)


display_insight_details(e23_df, 3)

## 6. Save Results

In [ ]:
# Save results
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
e23_df.to_csv(f"e23_gemini_pro_zero_shot_insights_{timestamp}.csv", index=False)

print(f"\n✓ Results saved: e23_gemini_pro_zero_shot_insights_{timestamp}.csv")
print(f"\nAverage Insight Quality Score: {e23_df['insight_quality_score'].mean():.3f}")

## Next Steps

1. Run E24 (Gemini Flash) and E25 (Llama) following the same pattern
2. Compare insight quality across models
3. Evaluate factual accuracy and actionability
4. Analyze correlation between sentiment and insight tone
5. Move to Few-Shot, Chain-of-Thought, and Tree-of-Thought strategies